<a href="https://colab.research.google.com/github/hannahscholz/PatternFlowHS/blob/topic-recognition/recognition/46616780_YOLO_Hannah_Scholz/YOLOV5_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training YOLOv5 on the ISIC dataset
---

#### Hannah Scholz
#### s4661678

---

Data is loaded in to Google Drive after executing the dataset.py file.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Make sure located in files not in another folder
%ls

drive/  sample_data/


In [3]:
# Clone the YOLOV5 model:

!git clone https://github.com/ultralytics/yolov5 
%cd yolov5
%pip install -qr requirements.txt  

import torch
from yolov5 import utils
display = utils.notebook_init() 

YOLOv5 🚀 v6.2-203-g6371de8 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)


Setup complete ✅ (12 CPUs, 83.5 GB RAM, 38.8/78.2 GB disk)


In [4]:
%cd ..
%ls


/content
drive/  yolov5/


In [ ]:
# Unzip the data
!unzip drive/MyDrive/ColabNotebooks/Archive.zip -d yolov5/data/

In [6]:
# Create file for parameters for the model

f = open("dataset.yaml", "a")

f.write("train: ../data/images/training/\n")
f.write("test: ../data/images/testing/\n")
f.write("val: ../data/images/validation/\n")
f.write("nc: 2\n")
f.write("names: ['melanoma', 'healthy']\n")
f.close()

In [7]:
%ls

dataset.yaml  drive/  yolov5/


In [8]:
%cd yolov5

/content/yolov5


**Training of YOLOV5 - Small Model**

In [ ]:
# 4 Epochs 
!python train.py --img 640 --batch 16 --epochs 10 --data ../dataset.yaml --weights yolov5s.pt

In [ ]:
# 50 Epochs
!python train.py --img 640 --batch 16 --epochs 50 --data ../dataset.yaml --weights yolov5s.pt

In [ ]:
# Epochs = 200(recommended)
!python train.py --img 640 --batch 16 --epochs 200 --data ../dataset.yaml --weights yolov5s.pt

In [ ]:
!nvidia-smi

In [ ]:
!python train.py --img 640 --batch 16 --epochs 300 --data ../dataset.yaml --weights yolov5s.pt

**Results using the Validation dataset for YOLOV5 - Small**

In [ ]:
# Using the validation dataset
!python detect.py --weights runs/train/exp/weights/best.pt --img 640 --conf 0.4 --source data/images/validation

In [ ]:
from IPython.display import display
from PIL import Image

image_path = "runs/detect/exp/ISIC_0001769.jpg"
display(Image.open(image_path))


**Training of YOLOV5 - Medium Model**

In [ ]:
#4 Epochs
!python train.py --img 640 --batch 16 --epochs 4 --data ../dataset.yaml --weights yolov5m.pt

In [ ]:
#50 Epochs
!python train.py --img 640 --batch 16 --epochs 50 --data ../dataset.yaml --weights yolov5m.pt

In [ ]:
#100 Epochs
!python train.py --img 640 --batch 16 --epochs 100 --data ../dataset.yaml --weights yolov5m.pt

In [ ]:
#300 Epochs
!python train.py --img 640 --batch 16 --epochs 300 --data ../dataset.yaml --weights yolov5m.pt

**Results using the Validation dataset for YOLOV5 - Medium**

In [ ]:
!python detect.py --weights runs/train/exp3/weights/best.pt --img 640 --conf 0.4 --source data/images/validation

In [ ]:
from IPython.display import display
from PIL import Image

image_path = "runs/detect/exp5/ISIC_0001769.jpg"
display(Image.open(image_path))

In [ ]:
# Validate YOLOv5s on validation
!python val.py --weights runs/train/exp3/weights/best.pt --data ../dataset.yaml --img 640 --half

**IOU Calculation**

In [ ]:
# Detect 
# Determine IOU
!python detect.py --source 'data/images/validation' --data ../dataset.yaml --weights 'runs/train/exp3/weights/best.pt' --conf 0.4 --iou 0.45 --augment

In [177]:
#IOU Function
def new_bbox_iou(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	
  # compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	
  # compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	
  # compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	
  # return the intersection over union value
	return iou


In [207]:
#This function calculates the IOU values for the testing dataset after training
import torch
import os
from os import listdir
from PIL import Image as im
import csv
import numpy as np

def IOU_calculator_testing(path_to_images, paths_to_labels):
  values_array = []

  # Model
  model = torch.hub.load('ultralytics/yolov5', 'custom', 'runs/train/exp3/weights/best.pt')  # change weights for specific epoch and model number required

  file_names = listdir(path_to_images)

  for filename in file_names:
      if filename.endswith(".DS_Store"):
          continue
      if filename.endswith(".ipynb_checkpoints"):
          continue

      with im.open(path_to_images + filename) as img:
        results = model(img)

        results.xywh[0]
        prediction = results.pandas().xywh[0]

        if prediction.empty: 
          continue

        xcenter = prediction.loc[0]["xcenter"]/640
        ycenter = prediction.loc[0]["ycenter"]/640
        width = prediction.loc[0]["width"]/640
        height = prediction.loc[0]["height"]/640

        label_file_name = filename.replace('.jpg', '.txt')

        values = open(paths_to_labels + label_file_name).read().split()
        xcenter2 = float(values[1])
        ycenter2 = float(values[2])
        width2 = float(values[3])
        height2 = float(values[4])

        iou = new_bbox_iou([xcenter, ycenter, width, height], [xcenter2, ycenter2, width2, height2])
        values_array.append(iou)
      
  average = sum(values_array) / len(values_array)
  return average


In [ ]:
av1 = IOU_calculator_testing("data/images/testing/", "data/labels/testing/")

av2 = IOU_calculator_testing("data/images/training/", "data/labels/training/")

av3 = IOU_calculator_testing("data/images/validation/", "data/labels/validation/")

total_avg = (av1+av2+av3)/3
print(total_avg)